# TRAINING AND TESTING SET CREATION

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 

In [ ]:
path_rawdata    = 'F://TFG//datasets/raw_datasets//'
path_train      = 'F://TFG//datasets//data_train//'
path_graphs     = 'F://TFG//graphs//'
path            = 'F:/TFG/datasets/nature-dataset/'

In [ ]:
data = pd.read_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';',index_col='wyId')
teams = pd.read_csv(path_rawdata+'teams.csv',sep=';')
raw_Data = pd.read_json(path_rawdata+'RAW_partidos.json')

In [ ]:
data = (pd.merge(data.reset_index(),raw_Data[['wyId','label']],how='left',on='wyId')
        .set_index('wyId'))

In [ ]:
data.shape

In [ ]:
data.loc[2499738]

In [ ]:
data.loc[[2501028, 2500919, 2576085, 2565922, 2500897, 2500016, 2576077, 2500047, 2565856,
 2516902, 2499874, 2499882, 2500940, 2517036, 2516833, 2516870, 2500082, 2576215,
 2499884, 2516891]].label

In [ ]:
data.columns

In [ ]:
data.to_csv(path_rawdata+'matches_wUltPartidos.csv',sep=';')

In [ ]:
results = pd.read_csv(path_graphs+'plot_results//train_preds_df.csv',sep=';')
results

We only want the data from the last 4 matches as training data.

In [ ]:
labels = []

for match in data.itertuples():
    if match.winner == 0: labels.append(0)
    elif match.winner == match.teamId_home: labels.append(1)
    else: labels.append(2)

# data['res'] = target
label_set = pd.DataFrame({'Labels':labels}, index=data.index)
label_set.head(3)

In [ ]:
training_features = ['mins4_H', 'mins4_A', 'shots_11H', 'shots_11A',
       'shots_acc_11H', 'shots_acc_11A', 'goals_H', 'goals_A', 'goals_ratio_H',
       'goals_ratio_A', 'passes_11H', 'passes_11A', 'passes_acc_11H',
       'passes_acc_11A', 'keyPasses_H', 'keyPasses_A', 'ataque_h', 'defensa_h',
       'ataque_a', 'defensa_a', 'ataquedefensa_H', 'ataquedefensa_A']

In [ ]:
X_df = data[training_features]
# X_df.to_csv(path_train+'training_features_DF.csv',sep=';')
X_df.head(5)

In [ ]:
y = label_set
X = data[training_features].dropna()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=1100, random_state=1
)

In [ ]:
X_train.head(5)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
X_train.to_csv(path_train+'X_train.csv',sep=';')
y_train.to_csv(path_train+'y_train.csv',sep=';')
X_test.to_csv(path_train+'X_test.csv',sep=';')
y_test.to_csv(path_train+'y_test.csv',sep=';')

In [ ]:
# np.save(path_train+'X_train', X_train)
# np.save(path_train+'y_train', y_train)
# np.save(path_train+'X_test', X_test)
# np.save(path_train+'y_test', y_test)

In [ ]:
# X_train = np.load(path_train+'X_train.npy')